In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

# Define paths to the dataset
train_dataset_path = '/Users/aishwarya/Jain University/Masters/Prodigy InfoTech/PRODIGY_ML_03/dogs-vs-cats/train'
test_dataset_path = '/Users/aishwarya/Jain University/Masters/Prodigy InfoTech/PRODIGY_ML_03/dogs-vs-cats/test'

# Image dimensions
IMG_SIZE = 224

# Load VGG16 model + higher level layers
base_model = VGG16(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)

def load_images_from_folder(folder, limit=500):
    images = []
    labels = []
    count = 0
    for img in os.listdir(folder):
        if count >= limit:
            break
        try:
            label = 'dog' if 'dog' in img else 'cat'  # Assuming the filename contains 'dog' or 'cat'
            img_path = os.path.join(folder, img)
            img_array = cv2.imread(img_path)
            img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
            img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            img_array = preprocess_input(img_array)
            features = model.predict(np.expand_dims(img_array, axis=0))[0]
            images.append(features)
            labels.append(label)
            count += 1
        except Exception as e:
            print(f"Error loading image {img}: {e}")
            continue
    return np.array(images), np.array(labels)

# Load a smaller subset of the dataset
X_train, y_train = load_images_from_folder(train_dataset_path, limit=500)  # Limit to 500 images
X_test, y_test = load_images_from_folder(test_dataset_path, limit=500)  # Limit to 500 images

# Check if images were loaded
if len(X_train) == 0 or len(y_train) == 0 or len(X_test) == 0 or len(y_test) == 0:
    print("No images were loaded. Please check the dataset path and structure.")
else:
    # Encode the labels
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    # Train the SVM model
    svm_model = SVC(kernel='linear')
    svm_model.fit(X_train, y_train_encoded)

    # Predict on the test set
    y_pred = svm_model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test_encoded, y_pred)
    print(f'Accuracy: {accuracy * 100:.2f}%')

    def classify_image(filepath):
        img_array = cv2.imread(filepath)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        img_array = preprocess_input(img_array)
        features = model.predict(np.expand_dims(img_array, axis=0))[0]
        prediction = svm_model.predict([features])
        return label_encoder.inverse_transform(prediction)[0]

    def upload_image():
        filepath = filedialog.askopenfilename()
        if filepath:
            image = Image.open(filepath)
            image = image.resize((250, 250), Image.ANTIALIAS)
            img = ImageTk.PhotoImage(image)
            panel = tk.Label(root, image=img)
            panel.image = img
            panel.grid(row=1, column=0, padx=10, pady=10)
            
            # Classify the image
            result = classify_image(filepath)
            result_label.config(text=f'Prediction: {result}')

    root = tk.Tk()
    root.title("Cat and Dog Classifier")

    upload_btn = tk.Button(root, text="Upload Image", command=upload_image)
    upload_btn.grid(row=0, column=0, padx=10, pady=10)

    result_label = tk.Label(root, text="Prediction: ")
    result_label.grid(row=2, column=0, padx=10, pady=10)

    root.mainloop()


2024-06-25 22:00:32.818646: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-25 22:00:47.146043: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 [==============================] - 1s 564ms/step


1/1 [==============================] - 1s 631ms/step


1/1 [==============================] - 0s 428ms/step


1/1 [==============================] - 1s 532ms/step


1/1 [==============================] - 1s 801ms/step


1/1 [==============================] - 1s 640ms/step


1/1 [==============================] - 1s 795ms/step
Accuracy: 48.20%
1/1 [==============================] - 1s 812ms/step
